# 自动向量化

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/jax/blob/main/docs/jax-101/03-vectorization.ipynb)

*Authors: Matteo Hessel*


## 手动量化（ Vectorization）

下面的例子是一维卷积

In [3]:
import jax
import jax.numpy as jnp

x = jnp.arange(5)
w = jnp.array([2., 3., 4.])

def convolve(x, w):
  output = []
  for i in range(1, len(x)-1):
    output.append(jnp.dot(x[i-1:i+2], w))
  return jnp.array(output)

convolve(x, w)

DeviceArray([11., 20., 29.], dtype=float32)

如果我们要对batch数据进行卷积，

In [4]:
xs = jnp.stack([x, x])
ws = jnp.stack([w, w])

最简单的方法就是写个for循环呗:

In [5]:
def manually_batched_convolve(xs, ws):
  output = []
  for i in range(xs.shape[0]):
    output.append(convolve(xs[i], ws[i]))
  return jnp.stack(output)

manually_batched_convolve(xs, ws)

DeviceArray([[11., 20., 29.],
             [11., 20., 29.]], dtype=float32)

但是效率太差。

怎么办，只能重新实现`convolve`，写一个支持批处理的版本：

In [6]:
def manually_vectorized_convolve(xs, ws):
  output = []
  for i in range(1, xs.shape[-1] -1):
    output.append(jnp.sum(xs[:, i-1:i+2] * ws, axis=1))
  return jnp.stack(output, axis=1)

manually_vectorized_convolve(xs, ws)

DeviceArray([[11., 20., 29.],
             [11., 20., 29.]], dtype=float32)

幸运的是，在JAX帮助下，你不必改动原来的函数就能支持批处理。

## Automatic Vectorization

`jax.vmap` 转换用于生成一个向量化版本的函数:

In [7]:
auto_batch_convolve = jax.vmap(convolve)

auto_batch_convolve(xs, ws)

DeviceArray([[11., 20., 29.],
             [11., 20., 29.]], dtype=float32)

同样，要对原函数进行tracing，自动为每个输入参数添加batch维度。

如果第一个维度不是batch维度，可以用 `in_axes` 和 `out_axes` 来制定输入和输出中batch轴的位置：

In [8]:
auto_batch_convolve_v2 = jax.vmap(convolve, in_axes=1, out_axes=1)

xst = jnp.transpose(xs)
wst = jnp.transpose(ws)

auto_batch_convolve_v2(xst, wst)

DeviceArray([[11., 11.],
             [20., 20.],
             [29., 29.]], dtype=float32)

`jax.vmap` 还支持只对函数的一个传参进行batch，比如对`convolve`函数，只想对`x`进行向量化，可以在 `in_axes` 中将第二个参数(`w`)设置为 `None`:

In [9]:
batch_convolve_v3 = jax.vmap(convolve, in_axes=[0, None])

batch_convolve_v3(xs, w)

DeviceArray([[11., 20., 29.],
             [11., 20., 29.]], dtype=float32)

## Combining transformations

JAX中的转换可以任意组合，:

In [10]:
jitted_batch_convolve = jax.jit(auto_batch_convolve)

jitted_batch_convolve(xs, ws)

DeviceArray([[11., 20., 29.],
             [11., 20., 29.]], dtype=float32)